# Libraries and data importing

Importing packages

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [206]:
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.statespace.structural import UnobservedComponents
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from statsmodels.tsa.holtwinters import Holt
from statsmodels.tsa.holtwinters import ExponentialSmoothing

In [2]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

Importing data

In [3]:
activities = pd.read_parquet('../data-raw/activities.parquet')
activities

,contributor,category,repository,activity,date
0,AmplabJenkins,bot,apache/spark,Commenting pull request,2022-11-25 09:55:19+00:00
1,AmplabJenkins,bot,apache/spark,Commenting pull request,2022-11-25 09:55:23+00:00
2,AmplabJenkins,bot,apache/spark,Commenting pull request,2022-11-25 09:55:26+00:00
3,analysis-bot,bot,facebook/react-native,Commenting pull request,2022-11-25 09:55:27+00:00
4,neos-bot,bot,neos/neos-ui-compiled,Pushing commits,2022-11-25 09:55:47+00:00
...,...,...,...,...,...
1015418,798388,human,879434,Reviewing code,2023-04-15 16:06:15+00:00
1015419,798388,human,879434,Reviewing code,2023-04-15 16:07:26+00:00
1015420,784775,human,643744,Creating branch,2023-04-15 16:07:33+00:00
1015421,784775,human,888378,Opening pull request,2023-04-15 16:08:07+00:00


Calculating the date 3 months ago from the last date in the data and filters the dataframe to include only the rows with dates greater than or equal to the date 3 months ago

In [4]:
thresholded_activities = (
    activities[activities['date'] >= activities['date'].max() - pd.DateOffset(months=3)]
    .groupby('contributor')
    .tail(300)
    .groupby(['category', pd.Grouper(key='date', freq='H'), 'contributor'])['activity']
    .count()
    .reset_index(name='n_activities')
    # keep data for contributors who have more than 10 different hour sequences
    .groupby('contributor')
    .filter(lambda x: len(x) >= 10)
)
thresholded_activities

,category,date,contributor,n_activities
0,bot,2023-01-15 16:00:00+00:00,AmplabJenkins,1
1,bot,2023-01-15 17:00:00+00:00,apollo-cla,1
2,bot,2023-01-15 17:00:00+00:00,stickler-ci[bot],1
3,bot,2023-01-15 18:00:00+00:00,AmplabJenkins,2
5,bot,2023-01-15 18:00:00+00:00,mysql-oca-bot,1
...,...,...,...,...
53322,human,2023-04-15 15:00:00+00:00,947579,6
53323,human,2023-04-15 16:00:00+00:00,387854,2
53324,human,2023-04-15 16:00:00+00:00,668373,1
53325,human,2023-04-15 16:00:00+00:00,784775,4


Filling n_activities with zeros for the empty hours between the minimum and maximum date for each contributor, and sets the category and contributor columns to a default values 

In [5]:
data = (
    thresholded_activities
    .groupby(['category','contributor'])
    .apply(
        lambda x: x
        .set_index('date')
        # resample data to fill in missing hours with zeros
        .resample('H')
        .sum()
        .fillna(0)
        .reset_index()
        # set the 'category' and 'contributor' columns to the values of the data group
        .assign(category=lambda y: y.iloc[0]['category'], contributor=lambda y: y.iloc[0]['contributor'])
        )
    .set_index('date')
    .rename_axis(None)
)
data

,category,contributor,n_activities
2023-01-18 10:00:00+00:00,bot,0crat,1
2023-01-18 11:00:00+00:00,bot,0crat,12
2023-01-18 12:00:00+00:00,bot,0crat,3
2023-01-18 13:00:00+00:00,bot,0crat,6
2023-01-18 14:00:00+00:00,bot,0crat,2
...,...,...,...
2023-04-06 05:00:00+00:00,human,999769,0
2023-04-06 06:00:00+00:00,human,999769,0
2023-04-06 07:00:00+00:00,human,999769,2
2023-04-06 08:00:00+00:00,human,999769,0


In [ ]:
data[data['contributor'] == 'pytorchbot']

# Autocorrelation function

In [ ]:
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [ ]:
temp = data[data['contributor'] == 'pytorchbot']
lags = min(int((len(temp)/2)-1),168)

acf_values, _ = acf(temp['n_activities'].values, nlags=lags, alpha=0.05)

print('The optimal numer of lags is :', np.argmax(acf_values[1:])+1)

# Plot the autocorrelation function
plot_acf(temp['n_activities'].values, lags=lags)
plt.show()

In [ ]:
temp = data[data['contributor'] == 'lingohub[bot]']
lags = min(int((len(temp)/2)-1),168)

acf_values, _ = pacf(temp['n_activities'].values, nlags=lags, alpha=0.05, method='ols')

print('The optimal numer of lags is :', np.argmax(acf_values[1:])+1)

# Plot the partial autocorrelation function
plot_pacf(temp['n_activities'].values, lags=lags)
plt.show()

# 1. Autoregressive model

Partial autocorrelation is a commonly used tool for identifying the order of an autoregressive model.

In [ ]:
def ar_model(contributor):

    # Spliting the data into training and testing sets for time series forecasting, using a time-based split with split size = 0.9
    train, test = (
        contributor.apply(lambda x: x[:int(0.9*len(x))]),
        contributor.apply(lambda x: x[int(0.9*len(x)):])
    )

    # Set the frequency of the index to hourly
    train.index.freq = 'H'

    # Choose the optimal number of lags
    lags = min(int((len(train)/2)-1),24*7)
    acf_values, _ = pacf(train['n_activities'].values, nlags=lags, alpha=0.05, method='ols')
    optimal_lags = np.argmax(acf_values[1:])+1

    # Fit the model
    model = AutoReg(train['n_activities'], lags=optimal_lags).fit()

    # Forecast the test set
    predictions = model.predict(start=len(train), end=len(train)+len(test)-1)

    # Create a series for evaluation metrics and sum of activities
    metrics = pd.Series({
        'contributor': contributor['contributor'].iloc[0],
        'category': contributor['category'].iloc[0],
        'r2': r2_score(test['n_activities'], predictions),
        'mae': mean_absolute_error(test['n_activities'], predictions),
        'mse': mean_squared_error(test['n_activities'], predictions),
        'n_activities': contributor['n_activities'].sum(),
        'optimal_lags': optimal_lags
    })

    return metrics


In [ ]:
# Apply the function to each contributor
ar_results = data.groupby(['category', 'contributor']).apply(ar_model).reset_index(drop=True)

In [ ]:
ar_results.sort_values(by='r2', ascending=False).head(40)

In [ ]:
ar_results.to_csv('../eval-rest/ar_model_metrics.csv', index=False)

# 2. Moving-average model

In [ ]:
def ma_model(contributor):

    print(contributor['contributor'].iloc[0])

    # Spliting the data into training and testing sets for time series forecasting, using a time-based split with split size = 0.9
    train, test = (
        contributor.apply(lambda x: x[:int(0.9*len(x))]),
        contributor.apply(lambda x: x[int(0.9*len(x)):])
    )

    # Set the frequency of the index to hourly
    train.index.freq = 'H'

    # Choose the optimal number of lags
    lags = min(int((len(train)/2)-1),24*7)
    acf_values, _ = acf(temp['n_activities'].values, nlags=lags, alpha=0.05)
    optimal_lags = np.argmax(acf_values[1:])+1

    # Fit the model
    model = ARIMA(train['n_activities'], order=(0, 0, optimal_lags)).fit()

    # Forecast the test set
    predictions = model.forecast(steps=len(test))

    # Create a series for evaluation metrics and sum of activities
    metrics = pd.Series({
        'contributor': contributor['contributor'].iloc[0],
        'category': contributor['category'].iloc[0],
        'r2': r2_score(test['n_activities'], predictions),
        'mae': mean_absolute_error(test['n_activities'], predictions),
        'mse': mean_squared_error(test['n_activities'], predictions),
        'n_activities': contributor['n_activities'].sum(),
        'optimal_lags': optimal_lags
    })

    return metrics

In [ ]:
# Apply the function to each contributor
ma_results = data.groupby(['category', 'contributor']).apply(ma_model).reset_index(drop=True)

In [ ]:
ma_results.sort_values(by='r2', ascending=False).head(40)

In [ ]:
ma_results.to_csv('../eval-rest/ma_model_metrics.csv', index=False)

# 3. Autoregressive Moving-average model

In [ ]:
def arma_model(contributor):

    print(contributor['contributor'].iloc[0])

    # Spliting the data into training and testing sets for time series forecasting, using a time-based split with split size = 0.9
    train, test = (
        contributor.apply(lambda x: x[:int(0.9*len(x))]),
        contributor.apply(lambda x: x[int(0.9*len(x)):])
    )

    # Set the frequency of the index to hourly
    train.index.freq = 'H'

    # Choose the optimal parameters
    lags = min(int((len(train)/2)-1),24*7)

    pacf_values, _ = pacf(temp['n_activities'].values, nlags=lags, alpha=0.05)
    p = np.argmax(pacf_values[1:])+1

    acf_values, _ = acf(temp['n_activities'].values, nlags=lags, alpha=0.05)
    q = np.argmax(acf_values[1:])+1

    print (p, q)

    # Fit the model
    model = ARIMA(train['n_activities'], order=(1, 0, 1)).fit()

    # Forecast the test set
    predictions = model.forecast(steps=len(test))

    # Create a series for evaluation metrics and sum of activities
    metrics = pd.Series({
        'contributor': contributor['contributor'].iloc[0],
        'category': contributor['category'].iloc[0],
        'r2': r2_score(test['n_activities'], predictions),
        'mae': mean_absolute_error(test['n_activities'], predictions),
        'mse': mean_squared_error(test['n_activities'], predictions),
        'n_activities': contributor['n_activities'].sum(),
        'p_order': p,
        'q_order': q
    })

    return print(metrics)

arma_model(data[data['contributor'] == 'lingohub[bot]'])

In [16]:
def arma_model(contributor):

    print(contributor['contributor'].iloc[0])

    # Spliting the data into training and testing sets for time series forecasting, using a time-based split with split size = 0.9
    train, test = (
        contributor.apply(lambda x: x[:int(0.9*len(x))]),
        contributor.apply(lambda x: x[int(0.9*len(x)):])
    )

    # Set the frequency of the index to hourly
    train.index.freq = 'H'

    # Fit the model
    model = ARIMA(train['n_activities'], order=(1, 0, 1)).fit(method='innovations_mle')

    # Forecast the test set
    predictions = model.forecast(steps=len(test))

    # Create a series for evaluation metrics and sum of activities
    metrics = pd.Series({
        'contributor': contributor['contributor'].iloc[0],
        'category': contributor['category'].iloc[0],
        'r2': r2_score(test['n_activities'], predictions),
        'mae': mean_absolute_error(test['n_activities'], predictions),
        'mse': mean_squared_error(test['n_activities'], predictions),
        'n_activities': contributor['n_activities'].sum()
    })

    return metrics

In [ ]:
# Apply the function to each contributor
arma_results = data.groupby(['category', 'contributor']).apply(arma_model).reset_index(drop=True)

In [18]:
arma_results.sort_values(by='r2', ascending=False).head(40)

,contributor,category,r2,mae,mse,n_activities
265,vscode-issue-tracker-bot,bot,1.000000,0.000000,0.000000,300
700,983874,human,0.186275,1.333177,4.223043,300
342,437693,human,0.178478,3.557969,14.990240,300
404,543595,human,0.097790,1.005003,2.078143,300
228,skia-flutter-autoroll,bot,0.069756,2.213537,7.947272,300
317,376887,human,0.051406,0.588989,0.859382,300
46,backportbot-nextcloud[bot],bot,0.051304,1.326977,2.682679,300
702,985588,human,0.046739,0.778744,1.766587,300
27,agones-bot,bot,0.022325,0.455257,0.299956,300
92,edx-requirements-bot,bot,0.021934,4.031566,35.355877,300


In [19]:
arma_results.to_csv('../eval-rest/arma_model_metrics.csv', index=False)

# 4. Autoregressive integrated Moving-average model

In [96]:
def arima_model(contributor):

    print(contributor['contributor'].iloc[0])

    # Spliting the data into training and testing sets for time series forecasting, using a time-based split with split size = 0.9
    train, test = (
        contributor.apply(lambda x: x[:int(0.9*len(x))]),
        contributor.apply(lambda x: x[int(0.9*len(x)):])
    )

    # Set the frequency of the index to hourly
    train.index.freq = 'H'

    # Fit the model
    model = ARIMA(train['n_activities'], order=(1, 1, 1)).fit()

    # Forecast the test set
    predictions = model.forecast(steps=len(test))

    # Create a series for evaluation metrics and sum of activities
    metrics = pd.Series({
        'contributor': contributor['contributor'].iloc[0],
        'category': contributor['category'].iloc[0],
        'r2': r2_score(test['n_activities'], predictions),
        'mae': mean_absolute_error(test['n_activities'], predictions),
        'mse': mean_squared_error(test['n_activities'], predictions),
        'n_activities': contributor['n_activities'].sum()
    })

    return metrics

In [ ]:
# Apply the function to each contributor
arima_results = data.groupby(['category', 'contributor']).apply(arima_model).reset_index(drop=True)

In [98]:
arima_results.sort_values(by='r2', ascending=False).head(40)

,contributor,category,r2,mae,mse,n_activities
265,vscode-issue-tracker-bot,bot,1.000000,0.000000,0.000000,300
234,sourcegraph-bot,bot,0.195866,0.330433,0.128661,300
317,376887,human,0.067154,0.501659,0.845115,300
702,985588,human,0.047634,0.778684,1.764928,300
404,543595,human,0.022110,1.021995,2.252464,300
141,johnpbloch-bot,bot,0.017387,0.291847,0.759658,300
688,978678,human,0.015407,1.572113,4.832544,300
16,PJBot,bot,0.012763,0.456669,0.544218,300
180,nur-bot,bot,0.011310,1.531511,3.341948,300
585,797989,human,0.008063,1.811103,4.524742,300


In [99]:
arima_results.to_csv('../eval-rest/arima_model_metrics.csv', index=False)

# 5. Seasonal Autoregressive integrated Moving-average model

In [ ]:
import pmdarima as pm

def sarimax_model(contributor):

    print(contributor['contributor'].iloc[0])

    # Spliting the data into training and testing sets for time series forecasting, using a time-based split with split size = 0.9
    train, test = (
        contributor.apply(lambda x: x[:int(0.9*len(x))]),
        contributor.apply(lambda x: x[int(0.9*len(x)):])
    )

    # Set the frequency of the index to hourly
    train.index.freq = 'H'

    # Use auto_arima to select the optimal seasonal_order parameter
    optimal_model = pm.auto_arima(train['n_activities'], seasonal=True, m=24)

    # Fit the model
    model = SARIMAX(train['n_activities'], order=optimal_model.order, seasonal_order=optimal_model.seasonal_order).fit(disp=False, method='lbfgs')

    # Forecast the test set
    predictions = model.predict(start=len(train), end=len(train)+len(test)-1)

    # Create a series for evaluation metrics and sum of activities
    metrics = pd.Series({
        'contributor': contributor['contributor'].iloc[0],
        'category': contributor['category'].iloc[0],
        'r2': r2_score(test['n_activities'], predictions),
        'mae': mean_absolute_error(test['n_activities'], predictions),
        'mse': mean_squared_error(test['n_activities'], predictions),
        'n_activities': contributor['n_activities'].sum(),
        'order' : optimal_model.order,
        'seasonal_order' : optimal_model.seasonal_order
    })

    return print(metrics)

sarimax_model(data[data['contributor'] == 'lingohub[bot]'])

In [157]:
def sarima_model(contributor):

    print(contributor['contributor'].iloc[0])

    # Spliting the data into training and testing sets for time series forecasting, using a time-based split with split size = 0.9
    train, test = (
        contributor.apply(lambda x: x[:int(0.9*len(x))]),
        contributor.apply(lambda x: x[int(0.9*len(x)):])
    )

    # Set the frequency of the index to hourly
    train.index.freq = 'H'

    # Fit the model
    model = SARIMAX(train['n_activities'], order=(1, 0, 1), seasonal_order=(1, 0, 1, 24), enforce_invertibility=False, enforce_stationarity=False).fit(disp=False, method='lbfgs')

    # Forecast the test set
    predictions = model.predict(start=len(train), end=len(train)+len(test)-1)

    # Create a series for evaluation metrics and sum of activities
    metrics = pd.Series({
        'contributor': contributor['contributor'].iloc[0],
        'category': contributor['category'].iloc[0],
        'r2': r2_score(test['n_activities'], predictions),
        'mae': mean_absolute_error(test['n_activities'], predictions),
        'mse': mean_squared_error(test['n_activities'], predictions),
        'n_activities': contributor['n_activities'].sum()
    })

    return metrics

In [ ]:
# Apply the function to each contributor
sarima_results = data.groupby(['category', 'contributor']).apply(sarima_model).reset_index(drop=True)

In [159]:
sarima_results.sort_values(by='r2', ascending=False).head(40)

,contributor,category,r2,mae,mse,n_activities
265,vscode-issue-tracker-bot,bot,1.000000,0.000000e+00,0.000000e+00,300
203,pxw-bot,bot,1.000000,6.301791e-07,1.196082e-11,180
181,octokit-fixture-user-a,bot,1.000000,3.858242e-06,4.052307e-10,300
7,Code-Inside-Bot,bot,0.997878,2.245156e-02,1.130119e-02,300
84,dontcallmedom-bot,bot,0.949912,1.855768e-01,5.241256e-01,300
204,pytorchbot,bot,0.913138,2.199351e-01,1.868500e-01,300
32,angular-automatic-lock-bot[bot],bot,0.893381,3.301472e-01,2.920989e+00,300
47,ballerina-bot,bot,0.812382,6.573424e-01,6.771834e-01,300
134,istio-policy-bot,bot,0.755105,2.764817e-01,2.205903e-01,300
259,usercont-release-bot,bot,0.749476,8.047926e-01,6.254830e+00,300


In [160]:
sarima_results.to_csv('../eval-rest/sarima_model_metrics.csv', index=False)

# 6. Unobserved components model

In [202]:
def uc_model(contributor):

    print(contributor['contributor'].iloc[0])

    # Spliting the data into training and testing sets for time series forecasting, using a time-based split with split size = 0.9
    train, test = (
        contributor.apply(lambda x: x[:int(0.9*len(x))]),
        contributor.apply(lambda x: x[int(0.9*len(x)):])
    )

    # Set the frequency of the index to hourly
    train.index.freq = 'H'

    # Fit the model
    model = UnobservedComponents(train['n_activities'], level=True, seasonal=24).fit(disp=False, method='lbfgs')

    # Forecast the test set
    predictions = model.predict(start=len(train), end=len(train)+len(test)-1)

    # Create a series for evaluation metrics and sum of activities
    metrics = pd.Series({
        'contributor': contributor['contributor'].iloc[0],
        'category': contributor['category'].iloc[0],
        'r2': r2_score(test['n_activities'], predictions),
        'mae': mean_absolute_error(test['n_activities'], predictions),
        'mse': mean_squared_error(test['n_activities'], predictions),
        'n_activities': contributor['n_activities'].sum()
    })

    return metrics

In [ ]:
# Apply the function to each contributor
uc_results = data.groupby(['category', 'contributor']).apply(uc_model).reset_index(drop=True)

In [204]:
uc_results.sort_values(by='r2', ascending=False).head(40)

,contributor,category,r2,mae,mse,n_activities
203,pxw-bot,bot,1.000000,2.565620e-14,8.115989e-27,180
181,octokit-fixture-user-a,bot,1.000000,1.387779e-15,8.896344e-30,300
7,Code-Inside-Bot,bot,0.996559,2.615370e-02,1.833045e-02,300
32,angular-automatic-lock-bot[bot],bot,0.891698,4.527253e-01,2.967123e+00,300
204,pytorchbot,bot,0.882490,1.421757e-01,2.527773e-01,300
259,usercont-release-bot,bot,0.720489,7.979506e-01,6.978526e+00,300
84,dontcallmedom-bot,bot,0.704575,5.645001e-01,3.091329e+00,300
47,ballerina-bot,bot,0.591159,7.010871e-01,1.475662e+00,300
175,nextcloud-bot,bot,0.436033,8.209108e+00,1.386004e+02,300
100,facebook-circleci-bot,bot,0.249652,5.874078e-02,5.190630e-02,160


In [205]:
uc_results.to_csv('../eval-rest/uc_model_metrics.csv', index=False)

# 9. Dynamic factor model

# 7. Simple exponential smoothing model

# 11. Holt’s linear exponential smoothing model

# 12. Holt-Winters exponential smoothing model